## musinsa_label 상품들의 좋아요 수 크롤링

In [ ]:
import pandas as pd

# 1. 파일 경로 지정
file_path_musinsa_label = "./Data/musinsa_label_data.csv"

# 2. CSV 파일을 DataFrame으로 불러오기
df_musinsa_label = pd.read_csv(file_path_musinsa_label, encoding="utf-8")  # 보통 utf-8, 혹은 cp949 필요할 수 있음

# 3. 데이터 확인
print(df_musinsa_label.head())

   goodsNo                                     goodsName displayGenderText  \
0  3046279              SELVEDGE DENIM 2ND JACKET_INDIGO                공용   
1  3046316  [MUT lab] 40S SELVEDGE STRAIGHT PANTS_INDIGO                공용   
2  3046300     30S CINCH BACK SELVEDGE WIDE PANTS_INDIGO                공용   
3  4693591    [MUT lab] SELVEDGE DENIM 1ST JACKET_INDIGO                남성   
4  5392266     TEA-CORE LEATHER W.W. II 1ST JACKET_BLACK                남성   

   normalPrice   price  couponPrice  saleRate  couponSaleRate  hasOptionPrice  \
0       159000  143100          NaN        10             NaN           False   
1       128000  115200          NaN        10             NaN           False   
2       128000  115200          NaN        10             NaN           False   
3       159000  143100          NaN        10             NaN           False   
4       388000  388000          NaN         0             NaN           False   

         brand  reviewCount  reviewScore  is

In [ ]:
# df_musinsa_label의 goodsNo 컬럼을 리스트로 변환
musinsa_label_goods_list = df_musinsa_label['goodsNo'].tolist()

In [3]:
# 리스트 요소 개수 확인
len(musinsa_label_goods_list)

2243

In [4]:
import requests
import pandas as pd

# 1. API URL
url = "https://like.musinsa.com/like/api/v2/liketypes/goods/counts"

# 2. 요청 헤더
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Origin": "https://www.musinsa.com",
    "Referer": "https://www.musinsa.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    # "Cookie": "여기에 최신 쿠키값 입력"
}

# 3. relationIds 
batch_size = 100
results_musinsa_label = []

# 100개 단위로 나눠서 API 호출
for i in range(0, len(musinsa_label_goods_list), batch_size):
    batch_musinsa_label = musinsa_label_goods_list[i:i + batch_size]
    payload = {
        "relationIds": batch_musinsa_label
    }
    
    response_musinsa_label = requests.post(url, headers=headers, json=payload)
    
    if response_musinsa_label.status_code == 200:
        data_musinsa_label = response_musinsa_label.json()
        try:
            items = data_musinsa_label["data"]["contents"]["items"]
            results_musinsa_label.extend(items)
        except KeyError:
            print("파싱 실패:", data_musinsa_label)
    else:
        print("요청 실패:", response_musinsa_label.status_code, response_musinsa_label.text)

# 4. 최종 DataFrame으로 변환
df_musinsa_counts = pd.DataFrame(results_musinsa_label)

In [ ]:
print('df_musinsa_counts 개수: ', len(df_musinsa_counts))

df_drawfit 개수:  2243


In [6]:
df_musinsa_counts

,likeType,relationId,count,liked
0,GOODS,3046279,14280,False
1,GOODS,3046316,8569,False
2,GOODS,3046300,13368,False
3,GOODS,4693591,4923,False
4,GOODS,5392266,1141,False
...,...,...,...,...
2238,GOODS,2723026,156,False
2239,GOODS,2450114,200,False
2240,GOODS,2450112,264,False
2241,GOODS,2450038,507,False


In [ ]:
# # csv 저장
# df_musinsa_counts.to_csv('Data/musinsa_counts_data.csv', index=False)